In [ ]:
!pip install torch  
!pip install torchaudio
!pip install transformers
!pip install pydub
!pip3 install youtube-search-python

In [ ]:
import numpy as np
import soundfile as sf
import librosa
import torch
import torchaudio


from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC
from IPython.display import (
    Javascript,
    Audio,
)

from youtubesearchpython import VideosSearch
from scipy.io import wavfile
from google.colab import output as colab_output
from io import BytesIO
from base64 import b64decode
from pydub import AudioSegment

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("Nonnyss/music-wav2vec2-th-finetune")
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("Nonnyss/music-wav2vec2-th-finetune")

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(seconds=5):
    display(Javascript(RECORD))
    print(f"Recording started for {seconds} seconds.")
    s = colab_output.eval_js("record(%d)" % (seconds * 1000))
    print("Recording ended.")
    b = b64decode(s.split(",")[1])

    fileformat = "wav"
    filename = f"_audio.{fileformat}"
    AudioSegment.from_file(BytesIO(b)).export(filename, format=fileformat)
    return filename

In [ ]:
file_name = record()
Audio(file_name)

<IPython.core.display.Javascript object>

Recording started for 5 seconds.
Recording ended.


In [ ]:
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
input_audio, _ = librosa.load(file_name, 
                              sr=16000)
input_values = tokenizer(input_audio, return_tensors="pt").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

videosSearch = VideosSearch(transcription, limit = 2)
result = videosSearch.result()
l = result.get('result')
print(l[0].get('title'))
print(l[0].get('link'))